<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Model-building" data-toc-modified-id="Model-building-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Model building</a></span></li><li><span><a href="#Predicting" data-toc-modified-id="Predicting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Predicting</a></span><ul class="toc-item"><li><span><a href="#Prediction-with-default-settings" data-toc-modified-id="Prediction-with-default-settings-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prediction with default settings</a></span></li><li><span><a href="#Prediction-with-balanced-class-weights" data-toc-modified-id="Prediction-with-balanced-class-weights-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Prediction with balanced class weights</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks")
sns.set_style("darkgrid")

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('sales.csv')
df.dropna(subset=['price'], inplace=True)
df.index = range(len(df))

In [3]:
df.head()

,cost,price,weight,purchase_date,product_type,product_level,maker,ingredient,height,width,depth
0,$333k,"$300,492",3 Ton 90 Kg,Dec 19 2008,"Q,B",advanced,M14122,"IN732052,IN732053",2.76 meters,97 cm,26 cm
1,NaN,"$430,570",3 Ton 30 Kg,Sep 10 1997,"J,D",basic,NaN,"IN732054,IN732055,IN732056,IN732057,IN732058",2.67 meters,98 cm,26 cm
2,$270k,"$213,070",3 Ton 40 Kg,Sep 05 2001,"J,D",basic,NaN,"IN732054,IN732059,IN732060",3.0 meters,93 cm,24 cm
3,NaN,"$229,174",3 Ton 50 Kg,Dec 23 2016,U,advanced,M14123,"IN732061,IN732062,IN732063",2.5 meters,102 cm,27 cm
4,$97k,"$122,659",2 Ton 970 Kg,Jan 12 2000,"D,R",advanced,NaN,"IN732064,IN732065,IN732066",2.47 meters,101 cm,26 cm


In [4]:
df['year'] = pd.to_datetime(df.purchase_date).dt.year
train_raw = df[df.year < 2015]
train_raw.index = range(len(train_raw))
test_raw = df[df.year >= 2015]
test_raw.index = range(len(test_raw))

# Model building

<font color='red'>Assignment:</font> Get **luxury** (**price** higher than 500k dollars) as targets for training and testing and assign them to variables **y_train** and **y_test**, respectively.

In [5]:
train_raw.head()

,cost,price,weight,purchase_date,product_type,product_level,maker,ingredient,height,width,depth,year
0,$333k,"$300,492",3 Ton 90 Kg,Dec 19 2008,"Q,B",advanced,M14122,"IN732052,IN732053",2.76 meters,97 cm,26 cm,2008
1,NaN,"$430,570",3 Ton 30 Kg,Sep 10 1997,"J,D",basic,NaN,"IN732054,IN732055,IN732056,IN732057,IN732058",2.67 meters,98 cm,26 cm,1997
2,$270k,"$213,070",3 Ton 40 Kg,Sep 05 2001,"J,D",basic,NaN,"IN732054,IN732059,IN732060",3.0 meters,93 cm,24 cm,2001
3,$97k,"$122,659",2 Ton 970 Kg,Jan 12 2000,"D,R",advanced,NaN,"IN732064,IN732065,IN732066",2.47 meters,101 cm,26 cm,2000
4,$59k,"$60,320",2 Ton 930 Kg,Aug 19 2011,H,super advanced,M14124,"IN732067,IN732068,IN732069,IN732070,IN732071,I...",2.71 meters,96 cm,25 cm,2011


In [6]:
def price2target(x):
    if type(x) == str:
        x = float(x.strip('$').replace(',',''))
    
    if x > 500000.0:
        return 1
    return 0

y_train = train_raw.price.map(price2target)
y_test =  test_raw.price.map(price2target)

<font color='red'>Assignment:</font> Build a transformation class to extract, preprocess, and transform **cost**; wrap up this class with **MinMaxScaler** and **LogisticRegression** to build a classification pipeline.

In [7]:
class Cost_Transformer(object):
    
    def fit(self, X, y=None):
        df = pd.DataFrame()
        df['cost'] = X.cost.map(lambda x: float(x.strip('$').strip('k'))*1000 if type(x)==str else x)
        self.median = df.median()
        
    def transform(self, X, y=None):
        df = pd.DataFrame()
        df['cost'] = X.cost.map(lambda x: float(x.strip('$').strip('k'))*1000 if type(x)==str else x)
        return df.fillna(self.median)
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

In [8]:
ct= Cost_Transformer()
X_train = ct.fit_transform(train_raw)
X_train

,cost
0,333000.0
1,122000.0
2,270000.0
3,97000.0
4,59000.0
...,...
2752,254000.0
2753,91000.0
2754,122000.0
2755,44000.0


In [9]:
y_train.isnull().sum()

0

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

steps = [('ct ',Cost_Transformer())
         ,('Rescale', MinMaxScaler())
         ,('Lgr', LogisticRegression())]
pipe= Pipeline(steps)

lgromodel=pipe.fit(train_raw,y_train)

<font color='red'>Question:</font> What is logistic regression? Is logistic regression a regression algorithm?

Logistic regression is a classifier using regression algorithm.

<font color='red'>Question:</font> Is logistic regression a linear algorithm? What is the relationship between logistic regression and linear regression?

Linear Regression is used to handle regression problems whereas Logistic regression is used to handle the classification problems. Linear regression provides a continuous output but Logistic regression provides discreet output.

Logistic regression is a classifier, comes from linear regression method.


<font color='red'>Question:</font> What is sigmoid function? What is it for?

y = 1/(1+e^(-z))

Sigmond function is a type'S' function. When variable is close to (+ float(inf)), y will be close to 1; when variable is close to (- float(inf)), y will be close to 0. It could be used to project all the data to range [0,1], make function fit for binary classificaition function.

# Predicting

## Prediction with default settings

<font color='red'>Assignment:</font> Use the **predict** function of the model to make predictions for the training set and test set, and assign the outputs to **y_train_pred** and **y_test_pred**, respectively.

In [11]:
y_train_pred = lgromodel.predict(train_raw)
y_test_pred = lgromodel.predict(test_raw)

<font color='red'>Question:</font> What are the shapes of **y_train_pred** and **y_test_pred**? What do the values in **y_train_pred** and **y_test_pred** mean?

In [12]:
y_train.shape

(2757,)

In [13]:
y_train_pred[y_train_pred==1].sum()

50

In [14]:
y_train_pred[y_train_pred==0].sum()

0

In [15]:
y_test.shape

(429,)

In [16]:
y_test_pred[y_test_pred==1].sum()

18

In [17]:
y_test_pred[y_test_pred==0].sum()

0

<font color='red'>Assignment:</font> Calculate the training and testing accuracy scores.

In [18]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, y_train_pred)

0.9390642002176278

In [19]:
accuracy_score(y_test, y_test_pred)

0.9114219114219114

<font color='red'>Assignment:</font> Calculate the precision, recall, and f1 scores. You can use classification report.

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_train, y_train_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2573
           1       0.66      0.18      0.28       184

    accuracy                           0.94      2757
   macro avg       0.80      0.59      0.63      2757
weighted avg       0.93      0.94      0.92      2757



In [21]:
print(classification_report(y_test, y_test_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95       383
           1       0.72      0.28      0.41        46

    accuracy                           0.91       429
   macro avg       0.82      0.63      0.68       429
weighted avg       0.90      0.91      0.89       429



<font color='red'>Assignment:</font> Get the confusion matrix.

In [22]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_train_pred)

array([[2556,   17],
       [ 151,   33]])

In [23]:
confusion_matrix(y_test, y_test_pred)

array([[378,   5],
       [ 33,  13]])

<font color='red'>Question:</font> Is this model better than the all-positive and all-negative models?

yes

##  Prediction with balanced class weights

<font color='red'>Assignment:</font> Set class_weight in Logistic Regression and retrain the model.

In [24]:
steps = [('ct ',Cost_Transformer())
         ,('Rescale', MinMaxScaler())
         ,('Lgr', LogisticRegression(class_weight = 'balanced'))]
pipe= Pipeline(steps)

lgr_bal_model=pipe.fit(train_raw,y_train)

y_train_pred = lgr_bal_model.predict(train_raw)
y_test_pred = lgr_bal_model.predict(test_raw)

<font color='red'>Assignment:</font> Re-calcualte all the above metrics and confusion matrix.

In [25]:
print('accuracy_score_train:',accuracy_score(y_train, y_train_pred))
print('accuracy_score_test:',accuracy_score(y_test, y_test_pred))


accuracy_score_train: 0.856728327892637
accuracy_score_test: 0.8694638694638694


In [26]:
print(classification_report(y_train, y_train_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92      2573
           1       0.28      0.72      0.40       184

    accuracy                           0.86      2757
   macro avg       0.63      0.79      0.66      2757
weighted avg       0.93      0.86      0.88      2757



In [27]:
print(classification_report(y_test, y_test_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.97      0.88      0.92       383
           1       0.44      0.76      0.56        46

    accuracy                           0.87       429
   macro avg       0.70      0.82      0.74       429
weighted avg       0.91      0.87      0.88       429



In [28]:
confusion_matrix(y_train, y_train_pred)

array([[2229,  344],
       [  51,  133]])

In [29]:
confusion_matrix(y_test, y_test_pred)

array([[338,  45],
       [ 11,  35]])

In [30]:
lgr_bal_model[-1].coef_

array([[9.29299333]])

<font color='red'>Question:</font> Do you see any difference? Why is the difference?

With balanced weight, accuracy, precision, and f1-score decreased, recall increased.

<font color='red'>Question:</font> Which results are more reasonable? Why?

Blanced. Weight of minor sample was adjusted.

<font color='red'>Question:</font> What are the other methods to deal with imbalanced data?

https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/

SMOTE